In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

In [ ]:
import { RecursiveSet, Tuple } from 'recursive-set';
type Variable = string;
type Literal  = Variable | Tuple<['¬', Variable]>;
type Clause   = RecursiveSet<Literal>;
type Clauses  = RecursiveSet<Clause>;

# Sudoku

In [ ]:
import * as DP from './06-Davis-Putnam';

The Finnish mathematician Arto Inkala claims to have created the [hardest sudoku](https://abcnews.go.com/blogs/headlines/2012/06/can-you-solve-the-hardest-ever-sudoku) ever.  It is defined below.

In [ ]:
function createPuzzle(): (number | string)[][] {
    return [
        [ 8 , '*', '*', '*', '*', '*', '*', '*', '*'],
        ['*', '*',  3,   6 , '*', '*', '*', '*', '*'],
        ['*',  7 , '*', '*',  9 , '*',  2 , '*', '*'],
        ['*',  5 , '*', '*', '*',  7 , '*', '*', '*'],
        ['*', '*', '*', '*',  4 ,  5 ,  7 , '*', '*'],
        ['*', '*', '*',  1 , '*', '*', '*',  3 , '*'],
        ['*', '*',  1 , '*', '*', '*', '*',  6 ,  8 ],
        ['*', '*',  8 ,  5 , '*', '*', '*',  1 , '*'],
        ['*',  9 , '*', '*', '*', '*',  4 , '*', '*']
    ];
}

We will solve this Sudoku using the Davis-Putnam algorithm.  We use the following variables:
* `Q<r,c,d>` is a Boolean variable stating that the field in row `r` and column `c` holds the digit `d`.
  Here, `r`, `c`, `d` are all elements from the set $\{1,\cdots,9\}$.
    
The function `varName(row, col, digit)` returns a formated string that is interpreted as a variable name.

In [ ]:
function varName(row: number, col: number, digit: number): string {
    return `Q<${row},${col},${digit}>`;
}

In [ ]:
varName(1,2,3);

The function `atMostOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at most one of the variables of `S` is true.

In [ ]:
function atMostOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    const result = new RecursiveSet<Clause>();
    for (const p of S) {
        for (const q of S) {
            if (p < q) {
                const lit1: Literal = new Tuple('¬', p);
                const lit2: Literal = new Tuple('¬', q);
                result.add(new RecursiveSet<Literal>(lit1, lit2));
            }
        }
    }
    return result;
}

The function `atLeastOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at least one of the variables of `S` is true.

In [ ]:
function atLeastOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    const clause = new RecursiveSet<Literal>();
    for (const v of S) {
        clause.add(v);
    }
    return new RecursiveSet<Clause>(clause);
}

The function `exactlyOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that exactly one of the variables of `S` is true.

In [ ]:
function exactlyOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    const atMost = atMostOne(S);
    const atLeast = atLeastOne(S);
    return atMost.union(atLeast);
}

In [ ]:
exactlyOne(new RecursiveSet<Variable>('a', 'b', 'c'));

The function `exactlyOnce` takes an array `L` of pairs of indices as its argument.  The elements of `L` are pairs of the form
`(row, col)`, where both `row` and `col` are elements of the set $\{1, \cdots, 9\}$.
It returns a set of formulas expressing that all Sudoku fields specified by the coordinate pairs in `L` take different digits as values.

In [ ]:
function exactlyOnce(L: Array<[number, number]>): RecursiveSet<Clause> {
    const Clauses = new RecursiveSet<Clause>();
    for (let digit = 1; digit <= 9; digit++) {
        const vars = new RecursiveSet<Variable>();
        for (const [row, col] of L) {
            vars.add(varName(row, col, digit));
        }
        const exact = exactlyOne(vars);
        for (const clause of exact) {
            Clauses.add(clause);
        }
    }
    return Clauses;
}

In [ ]:
const result = exactlyOnce(
    Array.from({ length: 9 }, (_, i) => [1, i + 1])
);

for (const clause of result) {
    console.log(clause.toString());
}

The function `exactlyOneDigit(row, col)` takes integers `row` and `col` as arguments.  These specify the row and column of a field in a Sudoku.  The function returns a set of clauses specifying that exactly one of the variables

* `Q<row,col,1>`, `Q<row,col,2>`, $\cdots$, `Q<row,col,9>`

is `true`.

In [ ]:
function exactlyOneDigit(row: number, col: number): RecursiveSet<Clause> {
    const vars = new RecursiveSet<Variable>();  
    for (let digit = 1; digit <= 9; digit++) {
        vars.add(varName(row, col, digit));
    }
    return exactlyOne(vars);
}

In [ ]:
for (const clause of exactlyOneDigit(1, 1)) {
    console.log(clause.toString()); 
}

The function `constraintsFromPuzzle`  returns a set of clauses stating that the variables corresponding to numbers that are already given in the Sudoku puzzle take the values that are specified.

In [ ]:
function constraintsFromPuzzle(): RecursiveSet<Clause> {
    const Puzzle = createPuzzle();
    const Clauses = new RecursiveSet<Clause>();
    for (let row = 0; row < 9; row++) {
        for (let col = 0; col < 9; col++) {
            const value = Puzzle[row][col];
            if (value !== '*') {
                const v = varName(row + 1, col + 1, value as number);
                const unitClause = new RecursiveSet<Literal>(v);
                Clauses.add(unitClause);
            }
        }
    }
    return Clauses;
}

In [ ]:
for (const clause of constraintsFromPuzzle()) {
    console.log(clause.toString()); 
}
constraintsFromPuzzle().size

The function `allConstraints` returns a CSP that encodes the given sudoku as a CSP.

In [ ]:
function allConstraints(): RecursiveSet<Clause> {
    const L = [1, 2, 3, 4, 5, 6, 7, 8, 9];
    // 1. Start with constraints from the puzzle
    let Clauses = constraintsFromPuzzle();
    // 2. There is exactly one digit in every field
    for (const row of L) {
        for (const col of L) {
            const digitConstraints = exactlyOneDigit(row, col);
            // Union nutzen ist effizienter als einzelne adds in Schleife
            // Hinweis: union gibt ein NEUES Set zurück, also zuweisen!
            Clauses = Clauses.union(digitConstraints);
        }
    }
    // 3. All entries in a row are unique
    for (const row of L) {
        const rowCells = L.map(col => [row, col] as [number, number]);
        Clauses = Clauses.union(exactlyOnce(rowCells));
    }
    // 4. All entries in a column are unique
    for (const col of L) {
        const colCells = L.map(row => [row, col] as [number, number]);
        Clauses = Clauses.union(exactlyOnce(colCells));
    }
    // 5. All entries in a 3x3 square are unique
    for (let r = 0; r < 3; r++) {
        for (let c = 0; c < 3; c++) {
            const blockCells: Array<[number, number]> = [];
            for (let row = 1; row <= 3; row++) {
                for (let col = 1; col <= 3; col++) {
                    blockCells.push([r * 3 + row, c * 3 + col]);
                }
            }
            Clauses = Clauses.union(exactlyOnce(blockCells));
        }
    }
    return Clauses;
}

In [ ]:
const clauses = allConstraints();
console.log("--- Clauses with size 1 ---");
for (const clause of clauses) {
    // Zugriff auf size Eigenschaft
    if (clause.size === 1) {
        console.log(clause.toString());
    }
}

In [ ]:
console.log("\n--- Clauses with size 9 ---");
for (const clause of clauses) {
    if (clause.size === 9) {
        console.log(clause.toString());
    }
}

In [ ]:
clauses.size

The function `solve(Constraints, Variables)` receives two arguments:
- `Constraints` is a set of formulas representing a constraint satisfaction problem.
- `Variables`   is the set of variables that occur in this formulas.

The function computes a solution to the given problem and returns this solution.

In [ ]:
function sudoku(): RecursiveSet<Clause> | null {
    const Clauses = allConstraints();
    const Solution = DP.solve(Clauses);
    const EmptyClause = new RecursiveSet<Literal>();
    if (!Solution.has(EmptyClause)) {
        return Solution;
    } else {
        console.log('The problem is not solvable!');
        return null;
    }
}

In [ ]:
console.time('sudoku');
const Solution = sudoku();
console.timeEnd('sudoku');

## Graphical Representation

In [ ]:
function arb<T extends Value>(S: RecursiveSet<T>): T | null {
    if (S.isEmpty()) {
        return null;
    }
    const val = S.pickRandom();
    return val !== undefined ? val : null;
}

In [ ]:
function transformSolution(Solution: RecursiveSet<Clause>): Record<string, number> {
    const Result: Record<string, number> = {};
    for (const UnitClause of Solution) {
        const literal = arb(UnitClause); 
        if (literal !== null && typeof literal === 'string') {
            const matches = literal.match(/(\d+),(\d+),(\d+)/);
            if (matches) {
                const row = parseInt(matches[1], 10);
                const col = parseInt(matches[2], 10);
                const digit = parseInt(matches[3], 10);
                Result[`V${row}${col}`] = digit;
            }
        }
    }
    return Result;
}

In [ ]:
import { display } from 'tslab';

function showSolution(Solution: RecursiveSet<Clause>, width: string = '50%'): void {
    const solutionMap: Record<string, number> = transformSolution(Solution);
    const Sudoku = createPuzzle();
    for (let row = 0; row < 9; row++) {
        for (let col = 0; col < 9; col++) {
            if (Sudoku[row][col] !== '*') {
                delete solutionMap[`V${row + 1}${col + 1}`];
            }
        }
    }
    let html = `<table style="width:${width}; border-collapse: collapse; border: 2px solid black; font-family: sans-serif;">`;
    for (let row = 0; row < 9; row++) {
        html += '<tr>';
        for (let col = 0; col < 9; col++) {
            const key = `V${row + 1}${col + 1}`;
            let value: number | string | undefined = solutionMap[key];
            const original = Sudoku[row][col];
            let cellStyle = "font-weight: normal;";
            
            if (original !== '*') {
                value = original;
                cellStyle = "font-weight: bold;";
            }
            
            const blockRow = Math.floor(row / 3);
            const blockCol = Math.floor(col / 3);
            const isGray = (blockRow + blockCol) % 2 !== 0;
            const bgColor = isGray ? '#f0f0f0' : '#ffffff';
            
            let borderStyle = "border: 1px solid #ccc;";
            if ((col + 1) % 3 === 0 && col < 8) borderStyle += "border-right: 2px solid black;";
            if ((row + 1) % 3 === 0 && row < 8) borderStyle += "border-bottom: 2px solid black;";
            
            html += `<td style="${borderStyle} width:30px; height:30px; text-align:center; font-size:20px; background-color:${bgColor}; ${cellStyle}">${value !== undefined ? value : ''}</td>`;
        }
        html += '</tr>';
    }
    html += '</table>';
    display.html(html);
}

In [ ]:
showSolution(Solution);